In [1]:
!pip install pyspark
!pip install findspark

Could not fetch URL https://pypi.org/simple/pyspark/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pyspark/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))) - skipping


ERROR: Could not find a version that satisfies the requirement pyspark (from versions: none)
ERROR: No matching distribution found for pyspark


In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import requests
import pandas as pd
import json
import time
import gzip

In [2]:
scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [3]:
# Usage of KafkaConsumer class
bootstrap_servers = 'localhost:9092'
topic_name = 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7'

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", bootstrap_servers)\
                .option("subscribe", topic_name)\
                .option("startingOffsets", "earliest").load()

In [4]:
selected_columns = ["rating", "reviewerName", "reviewText", "categories", "gPlusPlaceId", "unixReviewTime", "reviewTime", "gPlusUserId"]
local_api_url = 'http://127.0.0.1:5000/api/ie212_o11_group7/endpoint'

In [5]:
import pickle
df_data_places = pd.read_pickle('..\data_result\data_places.pkl')

In [9]:
from time import sleep
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").getOrCreate()

# Định nghĩa schema cho dữ liệu JSON
json_schema = StructType([
    StructField("rating", DoubleType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("categories", ArrayType(StringType()), True),
    StructField("gPlusPlaceId", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("gPlusUserId", StringType(), True)
])

# Đọc dữ liệu từ Kafka topic và chuyển đổi thành DataFrame
kafka_stream_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", topic_name) \
    .load()

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", bootstrap_servers)\
                .option("subscribe", topic_name)\
                .option("startingOffsets", "earliest").load()

# Chuyển đổi giá trị từ JSON string sang struct với schema đã định nghĩa
json_stream_df = kafkaDf.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", json_schema).alias("data")) \
    .select("data.*")

x = 0
while True:
    try:
        json_stream_df_pd = json_stream_df.toPandas()
        joined_df = pd.merge(json_stream_df_pd, df_data_places, on='gPlusPlaceId', how='inner')
        # joined_df = joined_df.drop(columns = ["reviewTime", "reviewerName", "gPlusUserId"])
        joined_df = joined_df.dropna()
        # joined_df = joined_df.reset_index().drop(columns = ["index"])
        if joined_df.empty:
            continue
        dict_from_df = joined_df.to_dict(orient='records')
        newest_review = dict_from_df[-1]


        # Cần xử lý lấy dòng đầu cuối cùng gửi cho API
        # json_stream_df_pd = json_stream_df.toPandas()
        # dict_from_df = json_stream_df_pd.to_dict(orient='records')
        # newest_review = dict_from_df[-1]

        # Add a timestamp to the review
        newest_review['last_update_time'] = time.time()
        
        # Convert the review to a JSON string
        message_value = json.dumps(newest_review)
        print(message_value)

        # Gửi request POST đến API local với dữ liệu JSON
        response = requests.post(local_api_url, json=message_value)

        if response.status_code == 200:
            x += 1
            print(f"Gửi dữ liệu {x} đã được đẩy lên thành công.")
        else:
            print("Đã có lỗi xảy ra:", response.status_code, response.text)

        sleep(2)
        # clear_output(wait=True)

    except KeyboardInterrupt:
        print("break")
        break

print("Live view ended...")


{"rating": 5.0, "reviewerName": "Mark Allen", "reviewText": "Incredible food in the oddest of locations...this place belongs in the Quarry rather than in a strip mall.  But I digress.  The burrata caprese speaks for itself and is the best item on the menu.  Personally, I think the appetizers are better than their pizzas, which are to die for.  Either go the Guy Fieri route and get the Pork Love or try the Arugula & Prosciutto pizza.  You won't be disappointed.", "categories": ["Pizza Restaurant", "Italian Restaurant", "Wine Bar"], "gPlusPlaceId": "100744538827780683773", "unixReviewTime": 1389045182.0, "reviewTime": "Jan 6, 2014", "gPlusUserId": "100014561186005202033", "name": "Dough Pizzeria Napoletana", "price": "$$", "address": ["6989 Blanco Rd", "San Antonio, TX 78216"], "phone": "(210) 979-6565", "gps": [29.519278, -98.507488], "last_update_time": 1706040706.6760194}
Gửi dữ liệu 1 đã được đẩy lên thành công.
{"rating": 5.0, "reviewerName": "Mark Allen", "reviewText": "Incredible 

In [54]:
json_stream_df_pd.head()

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId
0,5.0,Savanna Bowman,"WHEN I MET HANK AND GRAHAM OF SUTLIFF & STOUT,...","[""Insurance Attorney"",""Personal Injury Attorne...",113939686253574180930,1.342777e+09,"Jul 20, 2012",100000415460821501051
1,5.0,midengineZ06,"Great place to find tools, prices are.better.",None,118342244559062929091,NaN,None,100000420304795534622
2,5.0,Will Marks,None,"[""Pool Hall""]",107791752192016184839,1.394933e+09,"Mar 15, 2014",100000674723125309237
3,2.0,donovan selman,None,"[""Venezuelan Restaurant"",""South American Resta...",109578840050990181132,1.364542e+09,"Mar 29, 2013",100000714779149505680
4,5.0,Alexandra Jelinek,A great CrossFit place. Everyone is super fri...,"[""Gym"",""Personal Trainer"",""Physical Fitness Pr...",110859805635758910873,1.361812e+09,"Feb 25, 2013",100000797415618195956


In [32]:
def transform_row_to_json(row):
    transformed_data = {
        "rating": row.rating,
        "reviewerName": row.reviewerName,
        "reviewText": row.reviewText,
        "categories": json.loads(row.categories),
        "gPlusPlaceId": row.gPlusPlaceId,
        "unixReviewTime": row.unixReviewTime,
        "reviewTime": row.reviewTime,
        "gPlusUserId": row.gPlusUserId
    }
    return json.dumps(transformed_data)

In [22]:
from time import sleep
import json
import requests
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").getOrCreate()

# Định nghĩa schema cho dữ liệu JSON
json_schema = StructType([
    StructField("rating", DoubleType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("gPlusPlaceId", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("gPlusUserId", StringType(), True)
])

# Đọc dữ liệu từ Kafka topic và chuyển đổi thành DataFrame
kafka_stream_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", topic_name) \
    .load()

# Chuyển đổi giá trị từ JSON string sang struct với schema đã định nghĩa
json_stream_df = kafka_stream_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", json_schema).alias("data")) \
    .select("data.*")

# Start a query to process the streaming data
query = json_stream_df.writeStream \
    .outputMode("append") \
    .foreachBatch(send_to_api(json_stream_df)) \  
    .start()

# Await the termination of the query
query.awaitTermination()


SyntaxError: unexpected character after line continuation character (546545764.py, line 38)

In [19]:
# Define the function to send data to the API
def send_to_api(df):
    # Get the latest review from the batch
    newest_review = df.tail(1).iloc[0].to_dict()
    newest_review['last_update_time'] = time.time()
    message_value = json.dumps(newest_review)

    # Send a POST request to the API
    response = requests.post("http://127.0.0.1:5000/api/ie212_o11_group7/endpoint", json=message_value)

    if response.status_code == 200:
        print(f"Gửi dữ liệu đã được đẩy lên thành công.")
    else:
        print("Đã có lỗi xảy ra:", response.status_code, response.text)

In [27]:
json_stream_df.show(100)

+------+----------------+-------------------------------------+--------------------+--------------------+--------------+------------+--------------------+
|rating|    reviewerName|                           reviewText|          categories|        gPlusPlaceId|unixReviewTime|  reviewTime|         gPlusUserId|
+------+----------------+-------------------------------------+--------------------+--------------------+--------------+------------+--------------------+
|   3.0|          an lam|                    Chất lượng tạm ổn| ["Giải Trí - Café"]|10810331438000420...|    1372686659| Jul 1, 2013|10000001081715426...|
|   5.0|    HALİL TURGUT|                 Wc si temiz duzen...| ["Turkish Cuisine"]|10219412824160874...|    1342870724|Jul 21, 2012|10000001350028553...|
|   5.0|      森田さとこ|何回も私は予定に休みがセルバに行っ...|["Fishing","Pond ...|10140985882817540...|    1390653513|Jan 25, 2014|10000002133684886...|
|   5.0|      森田さとこ|今度は予定に休みが登米市に行きたい...|          ["Museum"]|10147717750015851...|    13891